In [74]:
import pandas as pd
import numpy as np

import requests
import io
from pypdf import PdfReader
pd.options.display.max_columns = None
import re

In [4]:
df = pd.read_csv("data/individual_market_medical.csv")
df.head(2)

,Unnamed: 0,State Code,FIPS County Code,County Name,Metal Level,Issuer Name,HIOS Issuer ID,Plan ID (Standard Component),Plan Marketing Name,Standardized Plan Option,Plan Type,Rating Area,Child Only Offering,Source,Customer Service Phone Number Local,Customer Service Phone Number Toll Free,Customer Service Phone Number TTY,Network URL,Plan Brochure URL,Summary of Benefits URL,Drug Formulary URL,Adult Dental,Child Dental,EHB Percent of Total Premium,Premium Scenarios,Premium Child Age 0-14,Premium Child Age 18,Premium Adult Individual Age 21,Premium Adult Individual Age 27,Premium Adult Individual Age 30,Premium Adult Individual Age 40,Premium Adult Individual Age 50,Premium Adult Individual Age 60,Premium Couple 21,Premium Couple 30,Premium Couple 40,Premium Couple 50,Premium Couple 60,"Couple+1 child, Age 21","Couple+1 child, Age 30","Couple+1 child, Age 40","Couple+1 child, Age 50","Couple+2 children, Age 21","Couple+2 children, Age 30","Couple+2 children, Age 40","Couple+2 children, Age 50","Couple+3 or more Children, Age 21","Couple+3 or more Children, Age 30","Couple+3 or more Children, Age 40","Couple+3 or more Children, Age 50","Individual+1 child, Age 21","Individual+1 child, Age 30","Individual+1 child, Age 40","Individual+1 child, Age 50","Individual+2 children, Age 21","Individual+2 children, Age 30","Individual+2 children, Age 40","Individual+2 children, Age 50","Individual+3 or more children, Age 21","Individual+3 or more children, Age 30","Individual+3 or more children, Age 40","Individual+3 or more children, Age 50",Standard Plan Cost Sharing,Medical Deductible - Individual - Standard,Drug Deductible - Individual - Standard,Medical Deductible - Family - Standard,Drug Deductible - Family - Standard,Medical Deductible - Family (Per Person) - Standard,Drug Deductible - Family (Per Person) - Standard,Medical Maximum Out Of Pocket - Individual - Standard,Drug Maximum Out Of Pocket - Individual - Standard,Medical Maximum Out Of Pocket - Family - Standard,Drug Maximum Out Of Pocket - Family - Standard,Medical Maximum Out Of Pocket - Family (Per Person) - Standard,Drug Maximum Out Of Pocket - Family (Per Person) - Standard,Primary Care Physician - Standard,Specialist - Standard,Emergency Room - Standard,Inpatient Facility - Standard,Inpatient Physician - Standard,Generic Drugs - Standard,Preferred Brand Drugs - Standard,Non-preferred Brand Drugs - Standard,Specialty Drugs - Standard,73 Percent Actuarial Value Silver Plan Cost Sharing,Medical Deductible - Individual - 73 Percent,Drug Deductible - Individual - 73 Percent,Medical Deductible - Family - 73 Percent,Drug Deductible - Family - 73 Percent,Medical Deductible - Family (Per Person) - 73 Percent,Drug Deductible - Family (Per Person) - 73 Percent,Medical Maximum Out Of Pocket - Individual - 73 Percent,Drug Maximum Out Of Pocket - Individual - 73 Percent,Medical Maximum Out Of Pocket - Family - 73 Percent,Drug Maximum Out Of Pocket - Family - 73 Percent,Medical Maximum Out Of Pocket - Family (Per Person) - 73 Percent,Drug Maximum Out Of Pocket - Family (Per Person) - 73 Percent,Primary Care Physician - 73 Percent,Specialist - 73 Percent,Emergency Room - 73 Percent,Inpatient Facility - 73 Percent,Inpatient Physician - 73 Percent,Generic Drugs - 73 Percent,Preferred Brand Drugs - 73 Percent,Non-preferred Brand Drugs - 73 Percent,Specialty Drugs - 73 Percent,87 Percent Actuarial Value Silver Plan Cost Sharing,Medical Deductible - Individual - 87 Percent,Drug Deductible - Individual - 87 Percent,Medical Deductible - Family - 87 Percent,Drug Deductible - Family - 87 Percent,Medical Deductible - Family (Per Person) - 87 Percent,Drug Deductible - Family (Per Person) - 87 Percent,Medical Maximum Out Of Pocket - Individual - 87 Percent,Drug Maximum Out Of Pocket - Individual - 87 Percent,Medical Maximum Out Of Pocket - Family - 87 Percent,Drug Maximum Out Of Pocket - Family - 87 Percent,Medical Maximum Out Of Pocket - Family (Per Person) - 87 Percent,Drug Maximum Out Of Pocket 

In [6]:
print(df.columns)

Index(['Unnamed: 0', 'State Code', 'FIPS County Code', 'County Name',
       'Metal Level', 'Issuer Name', 'HIOS Issuer ID',
       'Plan ID (Standard Component)', 'Plan Marketing Name',
       'Standardized Plan Option',
       ...
       'Drug Maximum Out Of Pocket - Family (Per Person) - 94 Percent',
       'Primary Care Physician - 94 Percent', 'Specialist - 94 Percent',
       'Emergency Room - 94 Percent', 'Inpatient Facility - 94 Percent',
       'Inpatient Physician - 94 Percent', 'Generic Drugs - 94 Percent',
       'Preferred Brand Drugs - 94 Percent',
       'Non-preferred Brand Drugs - 94 Percent',
       'Specialty Drugs - 94 Percent'],
      dtype='object', length=150)


In [8]:
df.columns = df.columns.str.replace(r'Premium Adult Individual Age ', 'Prem Single ', regex=True)
df.columns = df.columns.str.replace(r'Premium Couple ', 'Prem Coup ', regex=True)
df.columns = df.columns.str.replace(r'Couple\+1 child, Age ', 'Coup1 ', regex=True)
df.columns = df.columns.str.replace(r'Couple\+2 children, Age ', 'Coup2 ', regex=True)

In [10]:
df_single = df.groupby(['State Code'])[['Prem Single 21', 'Prem Single 30 ', 'Prem Single 40 ', 'Prem Single 50 ', 'Prem Single 60 ']].median().reset_index()
df_single.head(2)

,State Code,Prem Single 21,Prem Single 30,Prem Single 40,Prem Single 50,Prem Single 60
0,AK,739.76,839.62,945.41,1321.20,2007.70
1,AL,438.10,497.23,559.87,782.42,1188.97


In [12]:
df_fam4 = df.groupby(['State Code'])[['Coup2 21', 'Coup2 30 ', 'Coup2 40 ', 'Coup2 50']].median().reset_index()
df_fam4.head(2)

,State Code,Coup2 21,Coup2 30,Coup2 40,Coup2 50
0,AK,2611.34,2811.06,3022.64,3774.22
1,AL,1432.56,1550.82,1676.10,2121.20


In [14]:
df_fam4_min = df.groupby(['State Code'])[['Coup2 21', 'Coup2 30 ', 'Coup2 40 ', 'Coup2 50']].min().reset_index()
df_fam4_min.head(2)

,State Code,Coup2 21,Coup2 30,Coup2 40,Coup2 50
0,AK,1760.00,1894.00,2036.00,2542.00
1,AL,698.28,755.94,817.02,1033.98


In [16]:
# join median income
state = pd.read_csv("data/states.csv")
income = pd.read_csv("data/med_income_2024.csv")
state_income = state.merge(income, how = "inner", on="State")
state_income = state_income[['Abbreviation', 'Income']]
state_income = state_income.rename(columns = {'Abbreviation':'State Code'})
state_income['Income'] = pd.to_numeric(state_income['Income'])
state_income.head(2)

,State Code,Income
0,AL,77363.0
1,AK,104081.0


In [18]:
df_fam4_inc = df_fam4.merge(state_income, how="inner", on = "State Code")
df_fam4_inc.head(2)

,State Code,Coup2 21,Coup2 30,Coup2 40,Coup2 50,Income
0,AK,2611.34,2811.06,3022.64,3774.22,104081.0
1,AL,1432.56,1550.82,1676.10,2121.20,77363.0


In [20]:
df_fam4_inc['yearly coup2 40'] = df_fam4_inc['Coup2 40 ']*12

df_fam4_inc['perc coup2 40'] = df_fam4_inc['yearly coup2 40'].div(df_fam4_inc['Income'])
df_fam4_inc.sort_values(by="perc coup2 40", ascending=False)
df_fam4_inc.head(2)

,State Code,Coup2 21,Coup2 30,Coup2 40,Coup2 50,Income,yearly coup2 40,perc coup2 40
0,AK,2611.34,2811.06,3022.64,3774.22,104081.0,36271.68,0.348495
1,AL,1432.56,1550.82,1676.10,2121.20,77363.0,20113.20,0.259985


In [22]:
df_couple = df.groupby(['State Code'])[['Prem Coup 30 ', 'Prem Coup 40 ', 'Prem Coup 50 ', 'Prem Coup 60 ']].median()

#for each row, calculate the decade over decade increase in premiums 

percent_increase_couple = df_couple.pct_change(axis='columns')
percent_increase_couple.head(2)

,Prem Coup 30,Prem Coup 40,Prem Coup 50,Prem Coup 60
State Code,,,,
AK,NaN,0.125997,0.397489,0.519603
AL,NaN,0.125978,0.397503,0.519606


In [24]:
df_30s = df.groupby(['State Code'])[['Prem Single 30 ','Prem Coup 30 ','Coup1 30 ']].median()

#for each row, calculate the decade over decade increase in premiums 

percent_increase_30s = df_30s.pct_change(axis='columns')
percent_increase_30s.head(2)

,Prem Single 30,Prem Coup 30,Coup1 30
State Code,,,
AK,NaN,1.0,0.337004
AL,NaN,1.0,0.279730


In [26]:
df_40s = df.groupby(['State Code'])[['Prem Single 40 ','Prem Coup 40 ','Coup1 40 ']].median()

#for each row, calculate the decade over decade increase in premiums 

percent_increase_40s = df_40s.pct_change(axis='columns')
percent_increase_40s.head(2)

,Prem Single 40,Prem Coup 40,Coup1 40
State Code,,,
AK,NaN,1.0,0.299293
AL,NaN,1.0,0.248433


In [150]:
def extract_text(url):
    headers = {"User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36"}
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        pdf_file = io.BytesIO(response.content)
        pdf_reader = PdfReader(pdf_file)
        
        number_of_pages = len(pdf_reader.pages)
        print(number_of_pages)
        # page=pdf_reader.pages[0]
        # text = page.extract_text()
        # return text
        text = ""
        for n in range(number_of_pages):
            page = pdf_reader.pages[n]
            t = page.extract_text()
            print(t)
            text = text + t
            print(text)
        return text
    except requests.exceptions.RequestException as e:
        return "error downloading"
    except Exception as e:
        return "error"

In [152]:
test_url = df['Plan Brochure URL'][450]
pdf_text = extract_text(test_url)
pdf_text

20
Healthier People are 
Happier PeopleTMAn independent licensee of the Blue Cross and Blue Shield Association.
Healthier People are 
Happier PeopleTMAn independent licensee of the Blue Cross and Blue Shield Association.
2How to Enroll in a Health Plan? ................................................................... 4
Rewards  ...................................................................................................... 4
Preventive Care  ............................................................................................ 5
Mental Health  ............................................................................................... 6
Doctor On Demand ..................................................................................... 6
Blue 365  ...................................................................................................... 6
Mobile Apps ................................................................................................ 7
Metal 

'Healthier People are \nHappier PeopleTMAn independent licensee of the Blue Cross and Blue Shield Association.2How to Enroll in a Health Plan? ................................................................... 4\nRewards  ...................................................................................................... 4\nPreventive Care  ............................................................................................ 5\nMental Health  ............................................................................................... 6\nDoctor On Demand ..................................................................................... 6\nBlue 365  ...................................................................................................... 6\nMobile Apps ................................................................................................ 7\nMetal Level Information  ................................................................................ 7\nI

In [82]:
#skip table of contents (see: row 450, pg 1)

def clean_text(my_text):
    my_text = my_text.lower()
    my_text = re.sub("\n", "", my_text)
    return my_text

clean_text(pdf_text)

'THE CARE YOU NEED, WHEN YOU NEED IT.From primary care to urgent care and virtual medicine, you have options with Ambetter Health.Your Primary Care Provider (PCP) is your first choiceYour PCP is your main doctor. They’re your first option for things like checkups, vaccinations, colds, fevers and ongoing issues like allergies and asthma. If you need a PCP, use our online search tool to find an in-network doctor near you. In-network means the doctor, pharmacy, or facility is included in the Ambetter Health network. Always use in-network services and providers to prevent additional costs.Care where & when you need it with  Ambetter Health Virtual 24/7 Care *Get convenient 24/7 access to in-network telehealth providers for non-emergency medical and behavioral health issues through Ambetter Health Virtual 24/7 Care. You can get medical advice, a diagnosis or a prescription by phone or video with less waiting. Urgent Care provides convenience for immediate careWhen you can’t get in to see yo

In [86]:
df['Plan Brochure URL'][450]

'https://www.alabamablue.com/pb/2025individual.pdf'

In [102]:
df['Summary of Benefits URL'][0]

'https://www.premera.com/documents/045632_2025.pdf'